In [2]:
import torch

image = torch.tensor([
    # R 채널
    [[0.1, 0.1, 0.1],
     [0.1, 0.6, 0.1], # 가운데만 0.6 (갈색의 R)
     [0.1, 0.1, 0.1]],
    # G 채널
    [[0.8, 0.8, 0.8],
     [0.8, 0.3, 0.8], # 가운데만 0.3 (갈색의 G)
     [0.8, 0.8, 0.8]],
    # B 채널
    [[0.1, 0.1, 0.1],
     [0.1, 0.1, 0.1], # 가운데만 0.1 (갈색의 B)
     [0.1, 0.1, 0.1]]
])

In [3]:
# 무작위 가중치 생성 (AI의 초기 상태)
w = torch.randn(3, 3, 3, requires_grad=True)
b = torch.randn(1, requires_grad=True) # 편향

# 1단계: 선형 연산 (전체 픽셀과 가중치를 곱하고 다 더함)
z = torch.sum(image * w) + b

In [4]:
# 2단계: 활성화 함수 (시그모이드)
# "이것은 갈색 고양이인가?"에 대한 확률(0~1)을 얻고 싶으니까요.
prediction = torch.sigmoid(z)

print(f"AI의 예측 확률: {prediction.item() * 100:.2f}%")

AI의 예측 확률: 5.16%


In [6]:
loss = (1.0 - prediction)**2
loss.backward()
w.grad

tensor([[[-0.0093, -0.0093, -0.0093],
         [-0.0093, -0.0557, -0.0093],
         [-0.0093, -0.0093, -0.0093]],

        [[-0.0743, -0.0743, -0.0743],
         [-0.0743, -0.0279, -0.0743],
         [-0.0743, -0.0743, -0.0743]],

        [[-0.0093, -0.0093, -0.0093],
         [-0.0093, -0.0093, -0.0093],
         [-0.0093, -0.0093, -0.0093]]])

In [ ]:
import torch

learning_rate = 0.5 # 얼마나 빨리 배울지 정하는 속도

brownImage = torch.tensor([
    # R 채널
    [[0.1, 0.1, 0.1],
     [0.1, 0.6, 0.1], # 가운데만 0.6 (갈색의 R)
     [0.1, 0.1, 0.1]],
    # G 채널
    [[0.8, 0.8, 0.8],
     [0.8, 0.3, 0.8], # 가운데만 0.3 (갈색의 G)
     [0.8, 0.8, 0.8]],
    # B 채널
    [[0.1, 0.1, 0.1],
     [0.1, 0.1, 0.1], # 가운데만 0.1 (갈색의 B)
     [0.1, 0.1, 0.1]]
])


w = torch.randn(3, 3, 3, requires_grad=True) # 초기 가중치 설정
b = torch.randn(1, requires_grad=True) # 초기 바이어스 설정


for i in range(10000):
    z = torch.sum(brownImage * w) + b # 선형 연산
    prediction = torch.sigmoid(z) # 활성화 함수로 곡선 형태로 변환
    loss = (1.0 - prediction)**2 # 예측값과 실제값의 차이를 계산
    loss.backward() # 역전파

    with torch.no_grad(): # 업데이트 하는 동안은 미분 기록을 잠시 끕니다.
        w -= w.grad * learning_rate
        b -= b.grad * learning_rate
        w.grad.zero_()
        b.grad.zero_()
        
    
print(w)

tensor([[[ 0.1672, -0.3859,  1.1415],
         [ 2.4326,  0.6414,  2.1284],
         [-1.4997,  0.2418, -0.7296]],

        [[ 0.3081,  0.0597,  0.7179],
         [-0.6763, -1.7758,  1.8787],
         [ 2.9598,  1.0449,  0.6239]],

        [[-1.0111, -0.9450, -0.7047],
         [ 0.9393,  1.0464,  0.1072],
         [ 0.8600, -0.9251, -0.5799]]], requires_grad=True)


In [42]:
import torch

learning_rate = 0.5

# [데이터 세팅]
# 1. 정답 데이터: 가운데가 갈색 (정답: 1.0)
brown_image = torch.tensor([
    [[0.1, 0.1, 0.1], [0.1, 0.6, 0.1], [0.1, 0.1, 0.1]], # R
    [[0.8, 0.8, 0.8], [0.8, 0.3, 0.8], [0.8, 0.8, 0.8]], # G
    [[0.1, 0.1, 0.1], [0.1, 0.1, 0.1], [0.1, 0.1, 0.1]]  # B
])

# 2. 오답 데이터: 전체가 초록색 (정답: 0.0)
green_image = torch.tensor([
    [[0.1, 0.1, 0.1], [0.1, 0.1, 0.1], [0.1, 0.1, 0.1]], # R (중앙도 낮음)
    [[0.8, 0.8, 0.8], [0.8, 0.8, 0.8], [0.8, 0.8, 0.8]], # G (중앙도 높음)
    [[0.1, 0.1, 0.1], [0.1, 0.1, 0.1], [0.1, 0.1, 0.1]]  # B
])

w = torch.randn(3, 3, 3, requires_grad=True)
b = torch.randn(1, requires_grad=True)

for i in range(5000):
    # --- 갈색 이미지 학습 (정답은 1) ---
    pred_brown = torch.sigmoid(torch.sum(brown_image * w) + b)
    loss_brown = (1.0 - pred_brown)**2
    
    # --- 초록색 이미지 학습 (정답은 0) ---
    pred_green = torch.sigmoid(torch.sum(green_image * w) + b)
    loss_green = (0.0 - pred_green)**2
    
    # 두 오차를 합산 (두 문제 다 맞춰야 하니까)
    total_loss = loss_brown + loss_green
    total_loss.backward()

    with torch.no_grad():
        w -= w.grad * learning_rate
        b -= b.grad * learning_rate
        w.grad.zero_()
        b.grad.zero_()

    if i % 1000 == 0:
        print(f"회차 {i}: 갈색일확률={pred_brown.item()*100:.2f}%, 초록일확률={pred_green.item()*100:.2f}%")

# 학습 완료 후 가중치의 R채널 확인
print("\n--- 학습 완료 후 R채널 가중치 (중앙값을 확인하세요) ---")
print(w[0])

회차 0: 갈색일확률=99.36%, 초록일확률=99.63%
회차 1000: 갈색일확률=94.91%, 초록일확률=5.05%
회차 2000: 갈색일확률=96.55%, 초록일확률=3.42%
회차 3000: 갈색일확률=97.23%, 초록일확률=2.75%
회차 4000: 갈색일확률=97.63%, 초록일확률=2.35%

--- 학습 완료 후 R채널 가중치 (중앙값을 확인하세요) ---
tensor([[ 1.3789e-01,  6.3724e-04, -1.0820e-01],
        [ 7.9576e-01,  8.1639e+00, -7.9787e-01],
        [ 1.6078e+00,  1.0052e+00, -9.6651e-01]], grad_fn=<SelectBackward0>)


In [ ]:
import torch
import torch.nn as nn # 뉴런/레이어 모음
import torch.optim as optim # 최적화 알고리즘 모음

# 1. 데이터 준비 (Flatten: 3x3x3 이미지를 27개 일렬로 펴서 입력합니다)
# 뉴런 하나는 보통 일렬로 된 입력을 받는 구조이기 때문입니다.
X = torch.stack([brown_image.flatten(), green_image.flatten()]) 
Y = torch.tensor([[1.0], [0.0]]) # 정답 라벨

# 2. 모델 설계 (추상화의 핵심)
# nn.Linear(입력 개수, 출력 개수) -> 내부적으로 w, b를 자동 생성합니다.
model = nn.Sequential(
    nn.Linear(27, 1), 
    nn.Sigmoid()
)

# 3. 오차 계산법과 업데이트 도구 설정
criterion = nn.MSELoss() # (1.0 - pred)**2 를 대신해주는 표준 함수
optimizer = optim.SGD(model.parameters(), lr=0.5) # 업데이트 비서 고용

# 4. 학습 루프
for i in range(5000):
    # Forward: 예측
    prediction = model(X)
    
    # Loss: 오차 계산
    loss = criterion(prediction, Y)
    
    # Backward & Update: 역전파 후 비서에게 업데이트 지시
    optimizer.zero_grad() # 비서야, 이전 기록 지워라
    loss.backward()      # 비서야, 책임 소재 파악해라
    optimizer.step()      # 비서야, 한 발짝 움직여라 (w -= w.grad * lr)

    if i % 1000 == 0:
        print(f"회차 {i}: Loss = {loss.item():.4f}")

# 결과 확인: 모델 내부의 가중치 꺼내보기
trained_w = list(model.parameters())[0].data.view(3, 3, 3)
print("\n--- 추상화 모델로 학습된 R채널 가중치 ---")
print(trained_w[0])